<a href="https://colab.research.google.com/github/michaelstern336/AccessCtl/blob/master/DTis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/train.csv') as f:
  a=f.read(5)
  print (a)

,cont


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, make_scorer, mean_squared_error
from sklearn.metrics import confusion_matrix as cm
from sklearn.model_selection import train_test_split, GridSearchCV
import itertools
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
# from sklearn.impute import Imputer
from sklearn.pipeline import Pipeline

In [ ]:
df=pd.read_csv('/content/drive/My Drive/train.csv')
df.head()

,Unnamed: 0,content,cyber_label,environmental_issue
0,0,All rights reserved. MA23-16258 988982046\n\nh...,NaN,0
1,1,Revisiting our purpose and/or values statement...,NaN,0
2,2,Amid ongoing strategic competition in a\nmulti...,NaN,0
3,3,"Source: PwC Pulse Survey, November 2, 2022: ba...",NaN,0
4,4,Executive Summary 2 \nAgeing and\nHealth Conce...,NaN,1


In [ ]:
df["cyber_label"] = df.cyber_label.astype(object)
cleanup = {"cyber_label": {"NaN": 0,}}
df.replace(cleanup, inplace=True)
df.head()

,Unnamed: 0,content,cyber_label,environmental_issue
0,0,All rights reserved. MA23-16258 988982046\n\nh...,NaN,0
1,1,Revisiting our purpose and/or values statement...,NaN,0
2,2,Amid ongoing strategic competition in a\nmulti...,NaN,0
3,3,"Source: PwC Pulse Survey, November 2, 2022: ba...",NaN,0
4,4,Executive Summary 2 \nAgeing and\nHealth Conce...,NaN,1


In [ ]:
# Rename the columns
df = df.rename(columns={
    'Unnamed: 0': 'A',
    'content': 'B',
    'cyber_label': 'C',
    'environmental_issue': 'D'
})

# Print the DataFrame to verify the changes
print(df)

         A                                                  B    C  D
0        0  All rights reserved. MA23-16258 988982046\n\nh...  NaN  0
1        1  Revisiting our purpose and/or values statement...  NaN  0
2        2  Amid ongoing strategic competition in a\nmulti...  NaN  0
3        3  Source: PwC Pulse Survey, November 2, 2022: ba...  NaN  0
4        4  Executive Summary 2\nAgeing and\nHealth Conce...  NaN  1
...    ...                                                ...  ... ..
1295  1295  Source: PwC Pulse Survey, November 2, 2022: ba...  NaN  0
1296  1296  Military-driven innovations in relevant fields...  NaN  0
1297  1297  , artificial\nintelligence, automation in all ...  NaN  0
1298  1298  Year-over-year cyberattacks continue to evolve...  1.0  0
1299  1299  In fact, close to 40% of survey respondents sa...  NaN  0

[1300 rows x 4 columns]


In [ ]:
# Create a DataFrame with 1301 rows and 4 columns
df1 = {
    'A': list(range(1, 1302)),
    'B': ['Text {}'.format(i) for i in range(1, 1302)],
    'C': [0, 1] * 650,  # Dummy values in column C
    'D': [1, 0] * 650   # Dummy values in column D
}

df1 = pd.DataFrame(df)

# Print the DataFrame
print(df1)

         A                                                  B    C  D
0        0  All rights reserved. MA23-16258 988982046\n\nh...  NaN  0
1        1  Revisiting our purpose and/or values statement...  NaN  0
2        2  Amid ongoing strategic competition in a\nmulti...  NaN  0
3        3  Source: PwC Pulse Survey, November 2, 2022: ba...  NaN  0
4        4  Executive Summary 2\nAgeing and\nHealth Conce...  NaN  1
...    ...                                                ...  ... ..
1295  1295  Source: PwC Pulse Survey, November 2, 2022: ba...  NaN  0
1296  1296  Military-driven innovations in relevant fields...  NaN  0
1297  1297  , artificial\nintelligence, automation in all ...  NaN  0
1298  1298  Year-over-year cyberattacks continue to evolve...  1.0  0
1299  1299  In fact, close to 40% of survey respondents sa...  NaN  0

[1300 rows x 4 columns]


In [ ]:
# Add the value 0 to the empty cells in column C
df1['C'] = df1['C'].fillna(0)

# Print the updated DataFrame
print(df1)


         A                                                  B    C  D
0        0  All rights reserved. MA23-16258 988982046\n\nh...  0.0  0
1        1  Revisiting our purpose and/or values statement...  0.0  0
2        2  Amid ongoing strategic competition in a\nmulti...  0.0  0
3        3  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0
4        4  Executive Summary 2\nAgeing and\nHealth Conce...  0.0  1
...    ...                                                ...  ... ..
1295  1295  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0
1296  1296  Military-driven innovations in relevant fields...  0.0  0
1297  1297  , artificial\nintelligence, automation in all ...  0.0  0
1298  1298  Year-over-year cyberattacks continue to evolve...  1.0  0
1299  1299  In fact, close to 40% of survey respondents sa...  0.0  0

[1300 rows x 4 columns]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
import spacy

In [ ]:
df2 = df1.copy()
df2['C'] = df1['C'].fillna(0)
print(df2)

         A                                                  B    C  D
0        0  All rights reserved. MA23-16258 988982046\n\nh...  0.0  0
1        1  Revisiting our purpose and/or values statement...  0.0  0
2        2  Amid ongoing strategic competition in a\nmulti...  0.0  0
3        3  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0
4        4  Executive Summary 2\nAgeing and\nHealth Conce...  0.0  1
...    ...                                                ...  ... ..
1295  1295  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0
1296  1296  Military-driven innovations in relevant fields...  0.0  0
1297  1297  , artificial\nintelligence, automation in all ...  0.0  0
1298  1298  Year-over-year cyberattacks continue to evolve...  1.0  0
1299  1299  In fact, close to 40% of survey respondents sa...  0.0  0

[1300 rows x 4 columns]


In [ ]:
# Create a function to evaluate conditions and assign values to column E
def combine_labels(row):
    if row['C'] == 1 and row['D'] == 0:
        return 0
    elif row['C'] == 1 and row['D'] == 1:
        return 1
    elif row['C'] == 0 and row['D'] == 0:
        return 0
    elif row['C'] == 0 and row['D'] == 1:
        return 1

# Apply the function to create column E
df2['combined_label'] = df2.apply(combine_labels, axis=1)

# Save the DataFrame to a new CSV file with the added column
df2.to_csv('your_output_file.csv', index=False)
print(df2)

         A                                                  B    C  D  \
0        0  All rights reserved. MA23-16258 988982046\n\nh...  0.0  0   
1        1  Revisiting our purpose and/or values statement...  0.0  0   
2        2  Amid ongoing strategic competition in a\nmulti...  0.0  0   
3        3  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0   
4        4  Executive Summary 2\nAgeing and\nHealth Conce...  0.0  1   
...    ...                                                ...  ... ..   
1295  1295  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0   
1296  1296  Military-driven innovations in relevant fields...  0.0  0   
1297  1297  , artificial\nintelligence, automation in all ...  0.0  0   
1298  1298  Year-over-year cyberattacks continue to evolve...  1.0  0   
1299  1299  In fact, close to 40% of survey respondents sa...  0.0  0   

      combined_label  
0                  0  
1                  0  
2                  0  
3                  0  
4       

In [ ]:
# Rename the columns
df2 = df2.rename(columns={'combined_label': 'E'
})

# Print the DataFrame to verify the changes
print(df2)

         A                                                  B    C  D  E
0        0  All rights reserved. MA23-16258 988982046\n\nh...  0.0  0  0
1        1  Revisiting our purpose and/or values statement...  0.0  0  0
2        2  Amid ongoing strategic competition in a\nmulti...  0.0  0  0
3        3  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0  0
4        4  Executive Summary 2\nAgeing and\nHealth Conce...  0.0  1  1
...    ...                                                ...  ... .. ..
1295  1295  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0  0
1296  1296  Military-driven innovations in relevant fields...  0.0  0  0
1297  1297  , artificial\nintelligence, automation in all ...  0.0  0  0
1298  1298  Year-over-year cyberattacks continue to evolve...  1.0  0  0
1299  1299  In fact, close to 40% of survey respondents sa...  0.0  0  0

[1300 rows x 5 columns]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re


porter_stemmer = PorterStemmer()
nltk.download('stopwords')
stopwords_list = stopwords.words('english')



vectorizer = TfidfVectorizer(max_df=0.85, max_features=None,
                                 strip_accents='unicode', lowercase=True, analyzer='word',
                                 stop_words=stopwords_list, token_pattern=r'(?u)\b\w+\b',
                                 ngram_range=(1, 3), use_idf=True, smooth_idf=True, sublinear_tf=True)

def stemming(text):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", text).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return ' '.join(words)

def vectorization(df2,field):
    tfidf_col = vectorizer.fit_transform(df2[field].values)
    dataframe[field+"_tfidf"]=tfidf_col

    return df2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
print(df2)

         A                                                  B    C  D  E  \
0        0  All rights reserved. MA23-16258 988982046\n\nh...  0.0  0  0   
1        1  Revisiting our purpose and/or values statement...  0.0  0  0   
2        2  Amid ongoing strategic competition in a\nmulti...  0.0  0  0   
3        3  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0  0   
4        4  Executive Summary 2\nAgeing and\nHealth Conce...  0.0  1  1   
...    ...                                                ...  ... .. ..   
1295  1295  Source: PwC Pulse Survey, November 2, 2022: ba...  0.0  0  0   
1296  1296  Military-driven innovations in relevant fields...  0.0  0  0   
1297  1297  , artificial\nintelligence, automation in all ...  0.0  0  0   
1298  1298  Year-over-year cyberattacks continue to evolve...  1.0  0  0   
1299  1299  In fact, close to 40% of survey respondents sa...  0.0  0  0   

                                         processed_text  
0     All rights reserved. MA

In [72]:
# Text processing
def vectorize_text_field(df2, content, components=32, prefix='tfidf_'):
    tfv = TfidfVectorizer(min_df=0.01, max_df=1.0, max_features=None,
                      strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
                      ngram_range=(1, 3), use_idf=True, smooth_idf=True, sublinear_tf=True)
    svd = TruncatedSVD(n_components=components, random_state=2020)

    tfidf_col = tfv.fit_transform(df2[content].values.astype('U'))

    svd_col = svd.fit_transform(tfidf_col)
    svd_col = pd.DataFrame(svd_col)
    svd_col = svd_col.add_prefix('{}{}_'.format(prefix, content))

    features = []
    features.append(svd_col)

    summary_df = pd.concat(features, axis=1)

    final_df = pd.concat([df2, summary_df], axis=1)


    return final_df

vect_df = vectorize_text_field(df2, 'content')
vect_df['type'] = vect_df['type'].astype('category')
vect_df = vect_df.dropna(subset=['type'])
vect_df[['type_cat']] = vect_df[['type']].apply(lambda x: x.cat.codes)

KeyError: 'content'

In [98]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Assuming df2 is your DataFrame with the 'combined_label' column
# Replace 'your_output_file.csv' with the actual file path or DataFrame variable
# Assuming you have a DataFrame 'df'
# Custom function to remove punctuation using regular expression
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

# Assuming df is your DataFrame with the added column 'E'
# Replace 'your_output_file_processed.csv' with the actual file path or DataFrame variable

# Sample DataFrame with 'B' and 'E' columns
data = {'B': ["This is a sample text.", "Another example.", "Text processing is important.", "NLP is fascinating!"],
        'E': [1, 0, 1, 0]}
df2 = pd.DataFrame(data)

# Tokenize, remove stop words, and perform stemming
def process_text(row):
    text_tokens = word_tokenize(row['B'])
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in text_tokens if word.lower() not in stop_words]
    ps = PorterStemmer()
    stemmed_tokens = [ps.stem(word) for word in filtered_tokens]
    return ' '.join(stemmed_tokens)

# Apply text processing function to 'B' column
df2['processed_text'] = df.apply(process_text, axis=1)

# Save the DataFrame to a new CSV file with the processed text column
df2.to_csv('your_output_file_processed.csv', index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [99]:
print(df2)

                               B  E  \
0         This is a sample text.  1   
1               Another example.  0   
2  Text processing is important.  1   
3            NLP is fascinating!  0   

                                      processed_text  
0  right reserv . ma23-16258 988982046 http : //w...  
1  revisit purpos and/or valu statement 0 % 12 % ...  
2  amid ongo strateg competit multi-polar world w...  
3  sourc : pwc puls survey , novemb 2 , 2022 : ba...  


In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

# Assuming df is your DataFrame with the added column 'combined_label'
# Replace 'your_output_file_tfidf_scaled.csv' with the actual file path or DataFrame variable

# Apply TF-IDF vectorization to column B
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['B'])
tfidf_scaled = MinMaxScaler(feature_range=(-1, 1)).fit_transform(tfidf_matrix.toarray())

# Create a DataFrame with TF-IDF scaled values
df_tfidf_scaled = pd.DataFrame(tfidf_scaled, columns=[f'tfidf_scaled_{i}' for i in range(tfidf_scaled.shape[1])])

# Concatenate the new DataFrame with the original DataFrame
df2 = pd.concat([df2, df_tfidf_scaled], axis=1)

# Save the DataFrame to a new CSV file with the scaled TF-IDF values
df2.to_csv('your_output_file_tfidf_scaled.csv', index=False)

In [101]:
print(df2)

                                  B    E  \
0            This is a sample text.  1.0   
1                  Another example.  0.0   
2     Text processing is important.  1.0   
3               NLP is fascinating!  0.0   
4                               NaN  NaN   
...                             ...  ...   
1295                            NaN  NaN   
1296                            NaN  NaN   
1297                            NaN  NaN   
1298                            NaN  NaN   
1299                            NaN  NaN   

                                         processed_text  tfidf_scaled_0  \
0     right reserv . ma23-16258 988982046 http : //w...            -1.0   
1     revisit purpos and/or valu statement 0 % 12 % ...            -1.0   
2     amid ongo strateg competit multi-polar world w...            -1.0   
3     sourc : pwc puls survey , novemb 2 , 2022 : ba...            -1.0   
4                                                   NaN            -1.0   
...                  

In [102]:
from transformers import BertTokenizer, BertModel
import torch

# Assuming df is your DataFrame with the added column 'combined_label'
# Replace 'your_output_file_bert.csv' with the actual file path or DataFrame variable

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Apply text processing based on the 'combined_label' column
def process_text_bert(row):
    # Check the value in the 'combined_label' column
    if row['E'] == 1:
        # Tokenize and get BERT embeddings for positive cases
        tokens = tokenizer(row['B'], return_tensors='pt', truncation=True)
        outputs = model(**tokens)
        embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
        return embeddings
    else:
        # Return None for negative cases
        return None

# Apply the BERT processing function to the 'B' column
df2['bert_embeddings'] = df2.apply(process_text_bert, axis=1)

# Additional processing based on 'combined_label' can be added here if needed

# Save the DataFrame to a new CSV file with the BERT embeddings
df2.to_csv('your_output_file_bert.csv', index=False)

In [103]:
print(df2)

                                  B    E  \
0            This is a sample text.  1.0   
1                  Another example.  0.0   
2     Text processing is important.  1.0   
3               NLP is fascinating!  0.0   
4                               NaN  NaN   
...                             ...  ...   
1295                            NaN  NaN   
1296                            NaN  NaN   
1297                            NaN  NaN   
1298                            NaN  NaN   
1299                            NaN  NaN   

                                         processed_text  tfidf_scaled_0  \
0     right reserv . ma23-16258 988982046 http : //w...            -1.0   
1     revisit purpos and/or valu statement 0 % 12 % ...            -1.0   
2     amid ongo strateg competit multi-polar world w...            -1.0   
3     sourc : pwc puls survey , novemb 2 , 2022 : ba...            -1.0   
4                                                   NaN            -1.0   
...                  

In [104]:
from sklearn.preprocessing import MinMaxScaler

# Assuming df is your DataFrame with the added column 'combined_label'
# Replace 'your_output_file_scaled.csv' with the actual file path or DataFrame variable

# Scale the BERT embeddings
def scale_bert_embeddings(embeddings):
    if embeddings is not None:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        scaled_embeddings = scaler.fit_transform(embeddings)
        return scaled_embeddings
    else:
        return None

In [105]:
print(df2)

                                  B    E  \
0            This is a sample text.  1.0   
1                  Another example.  0.0   
2     Text processing is important.  1.0   
3               NLP is fascinating!  0.0   
4                               NaN  NaN   
...                             ...  ...   
1295                            NaN  NaN   
1296                            NaN  NaN   
1297                            NaN  NaN   
1298                            NaN  NaN   
1299                            NaN  NaN   

                                         processed_text  tfidf_scaled_0  \
0     right reserv . ma23-16258 988982046 http : //w...            -1.0   
1     revisit purpos and/or valu statement 0 % 12 % ...            -1.0   
2     amid ongo strateg competit multi-polar world w...            -1.0   
3     sourc : pwc puls survey , novemb 2 , 2022 : ba...            -1.0   
4                                                   NaN            -1.0   
...                  

In [106]:
X =  df2.drop('E', axis = 1)
Y = df2['E']
# print(X)
# print(Y)

In [107]:
print(X)
print(Y)

                                  B  \
0            This is a sample text.   
1                  Another example.   
2     Text processing is important.   
3               NLP is fascinating!   
4                               NaN   
...                             ...   
1295                            NaN   
1296                            NaN   
1297                            NaN   
1298                            NaN   
1299                            NaN   

                                         processed_text  tfidf_scaled_0  \
0     right reserv . ma23-16258 988982046 http : //w...            -1.0   
1     revisit purpos and/or valu statement 0 % 12 % ...            -1.0   
2     amid ongo strateg competit multi-polar world w...            -1.0   
3     sourc : pwc puls survey , novemb 2 , 2022 : ba...            -1.0   
4                                                   NaN            -1.0   
...                                                 ...             ...   
1295  

In [108]:
X = X.apply(pd.to_numeric, errors='coerce')
Y = Y.apply(pd.to_numeric, errors='coerce')
X.fillna(0, inplace=True)
Y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42)

In [109]:
print(X_train)
print(y_train)

        B  processed_text  tfidf_scaled_0  tfidf_scaled_1  tfidf_scaled_2  \
10    0.0             0.0            -1.0            -1.0            -1.0   
147   0.0             0.0            -1.0            -1.0            -1.0   
342   0.0             0.0            -1.0            -1.0            -1.0   
999   0.0             0.0            -1.0            -1.0            -1.0   
811   0.0             0.0            -1.0            -1.0            -1.0   
...   ...             ...             ...             ...             ...   
1095  0.0             0.0            -1.0            -1.0            -1.0   
1130  0.0             0.0            -1.0            -1.0            -1.0   
1294  0.0             0.0            -1.0            -1.0            -1.0   
860   0.0             0.0            -1.0            -1.0            -1.0   
1126  0.0             0.0            -1.0            -1.0            -1.0   

      tfidf_scaled_3  tfidf_scaled_4  tfidf_scaled_5  tfidf_scaled_6  \
10 

In [123]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB().fit(X_train,y_train)
# model.fit(X_train, y_train)

# y_pred_train = model.predict(X_test)
# y_pred = model.predict(X_test)
r = model.score(X_test, y_test)
print(r)

0.9923076923076923


In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB().fit(X_train,y_train)
# model.fit(X_train, y_train)

# y_pred_train = model.predict(X_test)
# y_pred = model.predict(X_test)
r = model.score(X_test, y_test)
print(r)

In [127]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier().fit(X_train, y_train)
r = model.score(X_test, y_test)
print(r)
print('accuracy =', model.score(X_test, y_test))

rf_preds = model.predict(X_test)

1.0
accuracy = 1.0


In [129]:
from sklearn import tree
from sklearn.metrics import accuracy_score
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
print(accuracy_score(predicted,y_test))

1.0


In [138]:
if 1:
    # Select relevant data and split into training and testing sets
    X = df2['B']
    Y = df2['E']
    labels = ['TRUE', 'FALSE']

    X_train, X_test, y_train, y_test = train_test_split(X,Y , train_size=0.8, random_state=42)

    # Prepare classifiers and params for Grid Search
    classifiers = [
        RandomForestClassifier()
        ]

    params_RFC = {
        'n_estimators': [100, 200, 300],
        'criterion': ['gini', 'entropy'],
        'n_jobs': [-1],
        'random_state': [0],
        }

    params = [
        params_RFC
        ]

    scorer = make_scorer(score_func=accuracy_score)

    for i in range(len(classifiers)):
        # Perform training over the parameters grid, cross-validate and evaluate on test set
        searcher = GridSearchCV(estimator=classifiers[i], param_grid=params[i],
                                scoring=scorer, n_jobs=-1, cv=10, verbose=10)
        searcher.fit(X_train, y_train)
        model = searcher.best_estimator_

        predictions_test = model.predict(X_test).astype(int)
        score_test = accuracy_score(y_test, predictions_test)
        predictions_train = model.predict(X_train).astype(int)
        score_train = accuracy_score(y_train, predictions_train)

        clf_report = classification_report(y_test, predictions_test)

        print(clf_report)

def plot_confusion_matrix (conf_matrix, labels_list, normalized=False, title = 'Confusion Matrix', cmap = plt.cm.Blues):
    if normalized:
        conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

    plt.imshow(conf_matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels_list))
    plt.xticks(tick_marks, labels_list, rotation=45)
    plt.yticks(tick_marks, labels_list)

    fmt = '.2f' if normalized else 'd'
    thresh = conf_matrix.max() / 2.
    for i, j in itertools.product(range(conf_matrix.shape[0]), range(conf_matrix.shape[1])):
        plt.text(j, i, format(conf_matrix[i, j], fmt), horizontalalignment='center',
                 color='white' if conf_matrix[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.savefig(os.path.join(FIGURES_LOCATION, title + '.png'), quality=100)
    plt.close()

  # Write the results to filE
# i = 0
# with open(os.path.join('RF Training Results.txt'), 'a') as f:
#             f.write('Classifier #{} - {}\n'.format(i+1, classifiers[i].__class__.__name__))
#             f.write('===================================\n')
#             f.write('Best train score: {:.5f}\n'.format(searcher.best_score_))
#             f.write('Best parameters: {}\n'.format(searcher.best_params_))
#             f.write('Score on test data: {:.5f}\n'.format(score_test))
#             f.write('Score on train data: {:.5f}\n'.format(score_train))
#             f.write('Classification report:\n{}\n\n\n'.format(clf_report))

#         # Compute and plot the Confusion Matrices for test and train data predictions
#             cm_test = cm(y_test, predictions_test)
#             plot_title = classifiers[i].__class__.__name__ + ' Test Data CM'
#             plot_confusion_matrix(cm_test, labels, title=plot_title)
#             cm_train = cm(y_train, predictions_train)
#             plot_title = classifiers[i].__class__.__name__ + ' Train Data CM'
#             plot_confusion_matrix(cm_train, labels, title=plot_title)

ValueError: Input y contains NaN.

In [137]:
i = 0
with open(os.path.join('RF Training Results.txt'), 'a') as f:
            f.write('Classifier #{} - {}\n'.format(i+1, classifiers[i].__class__.__name__))
            f.write('===================================\n')
            f.write('Best train score: {:.5f}\n'.format(searcher.best_score_))
            f.write('Best parameters: {}\n'.format(searcher.best_params_))
            f.write('Score on test data: {:.5f}\n'.format(score_test))
            f.write('Score on train data: {:.5f}\n'.format(score_train))
            f.write('Classification report:\n{}\n\n\n'.format(clf_report))

#         # Compute and plot the Confusion Matrices for test and train data predictions
            cm_test = cm(y_test, predictions_test)
            plot_title = classifiers[i].__class__.__name__ + ' Test Data CM'
            plot_confusion_matrix(cm_test, labels, title=plot_title)
            cm_train = cm(y_train, predictions_train)
            plot_title = classifiers[i].__class__.__name__ + ' Train Data CM'
            plot_confusion_matrix(cm_train, labels, title=plot_title)

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
import xgboost as xgb

model = xgb.XGBClassifier()
# clf.fit(X_train, y_train)
# print('accuracy =', clf.score(X_test, y_test))
# print(classification_report(y_test, clf.predict(X_test)))

parameters = {'nthread':[4],
              'objective':['multi:softmax'],
              'eta': np.arange(0.1, 0.26, 0.05),
              'num_class': [2],
              'learning_rate': [0.03, 0.05, 0.07],
              'max_depth': [6, 7, 8],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5, 10, 50, 100],
              'missing':[-999],
              'seed': [1337]}


clf = GridSearchCV(model, parameters, n_jobs=5,
                   cv=StratifiedKFold(n_splits=5, shuffle=True).split(X_train, y_train),
                   scoring='f1_weighted',
                   verbose=2, refit=True)

clf.fit(X_train, y_train)


best_pars = clf.best_params_

print('accuracy =', clf.best_estimator_.score(X_test, y_test))
print(classification_report(y_test, clf.best_estimator_.predict(X_test)))

xgb_preds = clf.predict(X_test)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [126]:
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve, plot_precision_recall_curve, classification_report

print(classification_report(y_test, model.predict(X_test)))

ImportError: cannot import name 'plot_precision_recall_curve' from 'sklearn.metrics' (/usr/local/lib/python3.10/dist-packages/sklearn/metrics/__init__.py)

In [121]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
# Train a Multinomial Naive Bayes classifier
model = MultinomialNB().fit(X_train, y_train)
r = model.score(X_test, y_test)
print(r)

    # Make predictions on the test set
predictions = model.predict(X_test)

    # Evaluate accuracy and show classification report
accuracy = accuracy_score(test_labels, predictions)
report = classification_report(test_labels, predictions)

print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

ValueError: Negative values in data passed to MultinomialNB (input X)

In [91]:
X =  df2['B']
Y = df2['E']
print(X)
print(Y)

0       All rights reserved. MA23-16258 988982046\n\nh...
1       Revisiting our purpose and/or values statement...
2       Amid ongoing strategic competition in a\nmulti...
3       Source: PwC Pulse Survey, November 2, 2022: ba...
4       Executive Summary 2\nAgeing and\nHealth Conce...
                              ...                        
1295    Source: PwC Pulse Survey, November 2, 2022: ba...
1296    Military-driven innovations in relevant fields...
1297    , artificial\nintelligence, automation in all ...
1298    Year-over-year cyberattacks continue to evolve...
1299    In fact, close to 40% of survey respondents sa...
Name: B, Length: 1300, dtype: object
0       0
1       0
2       0
3       0
4       1
       ..
1295    0
1296    0
1297    0
1298    0
1299    0
Name: E, Length: 1300, dtype: int64
